In [3]:
import math
import gurobipy as gp
from gurobipy import GRB

# Define the Sudoku input file
filename = "sudoku.txt"  

# Read the Sudoku puzzle from the file
with open(filename, "r") as f:
    grid = f.read().split()

n = len(grid[0])  # Size of the grid (typically 9 for a standard Sudoku)
s = int(math.sqrt(n))  # Size of sub-grids (3 for a standard Sudoku)

# Create the optimization model
model = gp.Model("sudoku")

# Add variables: x[i, j, v] is 1 if cell (i, j) contains the value v+1
vars = model.addVars(n, n, n, vtype=GRB.BINARY, name="G")

# Fix variables for pre-filled cells
for i in range(n):
    for j in range(n):
        if grid[i][j] != ".":
            v = int(grid[i][j]) - 1
            vars[i, j, v].LB = 1  

# Add constraints
# Each cell must take exactly one value
model.addConstrs(
    (vars.sum(i, j, "*") == 1 for i in range(n) for j in range(n)), name="V"
)

# Each value must appear exactly once in each row
model.addConstrs(
    (vars.sum(i, "*", v) == 1 for i in range(n) for v in range(n)), name="R"
)

# Each value must appear exactly once in each column
model.addConstrs(
    (vars.sum("*", j, v) == 1 for j in range(n) for v in range(n)), name="C"
)

# Each value must appear exactly once in each sub-grid
model.addConstrs(
    (
        gp.quicksum(
            vars[i, j, v]
            for i in range(i0 * s, (i0 + 1) * s)
            for j in range(j0 * s, (j0 + 1) * s)
        )
        == 1
        for v in range(n)
        for i0 in range(s)
        for j0 in range(s)
    ),
    name="Sub",
)

# Optimize the model
model.optimize()

# Print the solution
print("\nSolution:\n")
solution = model.getAttr("X", vars)

for i in range(n):
    sol = ""
    for j in range(n):
        for v in range(n):
            if solution[i, j, v] > 0.5:
                sol += str(v + 1)
    print(sol)


Set parameter Username
Academic license - for non-commercial use only - expires 2025-06-26
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (26100.2))

CPU model: AMD Ryzen 7 5800HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 324 rows, 729 columns and 2916 nonzeros
Model fingerprint: 0xe49e088a
Variable types: 0 continuous, 729 integer (729 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 324 rows and 729 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+0